In [1]:
import pandas as pd
import numpy as np
import inflect
import itertools
import splitter
import string
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pandarallel import pandarallel
import re
pandarallel.initialize(progress_bar=True)
p = inflect.engine()


INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
# Loading dataset
data = pd.read_csv(
    "../original-data/IEEE VIS papers 1990-2020 - Main dataset.csv")
data


,Conference,Year,Title,DOI,Link,FirstPage,LastPage,PaperType,Abstract,AuthorNames-Deduped,AuthorNames,AuthorAffiliation,InternalReferences,AuthorKeywords,AminerCitationCount_04-2020,XploreCitationCount - 2021-02,PubsCited,Award
0,InfoVis,2011,D³ Data-Driven Documents,10.1109/TVCG.2011.185,http://dx.doi.org/10.1109/TVCG.2011.185,2301,2309,J,Data-Driven Documents (D3) is a novel represen...,Michael Bostock;Vadim Ogievetsky;Jeffrey Heer,Michael Bostock;Vadim Ogievetsky;Jeffrey Heer,NaN,10.1109/INFVIS.2000.885091;10.1109/INFVIS.2000...,"Information visualization, user interfaces, to...",1537.0,1197.0,41.0,NaN
1,Vis,1991,Tree-maps: a space-filling approach to the vis...,10.1109/VISUAL.1991.175815,http://dx.doi.org/10.1109/VISUAL.1991.175815,284,291,C,A method for visualizing hierarchically struct...,Brian Johnson;Ben Shneiderman,B. Johnson;B. Shneiderman,"Dept. of Comput. Sci., Maryland Univ., College...",NaN,NaN,1132.0,423.0,23.0,NaN
2,Vis,1990,Parallel coordinates: a tool for visualizing m...,10.1109/VISUAL.1990.146402,http://dx.doi.org/10.1109/VISUAL.1990.146402,361,378,C,A methodology for visualizing analytic and syn...,Alfred Inselberg;Bernard Dimsdale,A. Inselberg;B. Dimsdale,"IBM Sci. Center, Los Angeles, CA, USA;IBM Sci....",NaN,NaN,963.0,373.0,47.0,NaN
3,InfoVis,2006,Hierarchical Edge Bundles: Visualization of Ad...,10.1109/TVCG.2006.147,http://dx.doi.org/10.1109/TVCG.2006.147,741,748,J,A compound graph is a frequently encountered t...,Danny Holten,Danny Holten,NaN,10.1109/INFVIS.2004.1;10.1109/INFVIS.2003.1249...,"Network visualization, edge bundling, edge agg...",700.0,507.0,33.0,TT;BP
4,Vis,1997,ROAMing terrain: Real-time Optimally Adapting ...,10.1109/VISUAL.1997.663860,http://dx.doi.org/10.1109/VISUAL.1997.663860,81,88,C,Terrain visualization is a difficult problem f...,Mark A. Duchaineau;Murray Wolinsky;David E. Si...,M. Duchaineau;M. Wolinsky;D.E. Sigeti;M.C. Mil...,"Los Alamos Nat. Lab., NM, USA",10.1109/VISUAL.1996.567600;10.1109/VISUAL.1996...,"triangle bintree, view-dependent mesh, frame-t...",615.0,207.0,19.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3389,VAST,2020,Visual Analytics of Multivariate Event Sequenc...,10.1109/VAST50239.2020.00009,http://dx.doi.org/10.1109/VAST50239.2020.00009,36,47,C,"In this work, we propose a generic visual anal...",Jiang Wu;Ziyang Guo;Zuobin Wang;Qingyang Xu;Yi...,Jiang Wu;Ziyang Guo;Zuobin Wang;Qingyang Xu;Yi...,"Zhejiang University,The State Key Lab of CAD&C...","10.1109/TVCG.2019.2934209,10.1109/TVCG.2017.27...","Sports Analytics,Event Sequence,Multivariate D...",NaN,0.0,55.0,NaN
3390,VAST,2020,Visual Causality Analysis of Event Sequence Data,10.1109/TVCG.2020.3030465,http://dx.doi.org/10.1109/TVCG.2020.3030465,1343,1352,J,Causality is crucial to understanding the mech...,Zhuochen Jin;Shunan Guo;Nan Chen;Daniel Weisko...,Zhuochen Jin;Shunan Guo;Nan Chen;Daniel Weisko...,iDVx Lab at Tongji University;iDVx Lab at Tong...,"10.1109/INFVIS.2003.1249025,10.1109/TVCG.2014....","Event sequence data,causality analysis,visual ...",NaN,0.0,53.0,NaN
3391,VAST,2020,Visual cohort comparison for spatial single-ce...,10.1109/TVCG.2020.3030336,http://dx.doi.org/10.1109/TVCG.2020.3030336,733,743,J,Spatially-resolved omics-data enable researche...,Antonios Somarakis;Marieke E. Ijsselsteijn;Sie...,Antonios Somarakis;Marieke E. Ijsselsteijn;Sie...,"Department of Radiology, Division of Image Pro...","10.1109/TVCG.2013.124,10.1109/TVCG.2018.286490...","Visual analytics,Imaging Mass Cytometry,Vectra...",NaN,0.0,58.0,NaN
3392,VAST,2020,Visual Neural Decomposition to Explain Multiva...,10.1109/TVCG.2020.3030420,http://dx.doi.org/10.1109/TVCG.2020.3030420,1374,1384,J,Investigating relationships between variables ...,Johannes Knittel;Andres Lalama;Steffen Koch;Th...,Johannes Knittel;Andres Lalama;Steffen Koch;Th...,University of Stuttgart;University of Stuttgar...,"10.1109/INFVIS.2004.68,10.1109/VAST.2008.46773...","Visual Analytics,Multivariate Data Analysis,Ma..."

In [3]:
# Removing papers with no keywords
data = data.loc[data['AuthorKeywords'].fillna('') != '']
data


,Conference,Year,Title,DOI,Link,FirstPage,LastPage,PaperType,Abstract,AuthorNames-Deduped,AuthorNames,AuthorAffiliation,InternalReferences,AuthorKeywords,AminerCitationCount_04-2020,XploreCitationCount - 2021-02,PubsCited,Award
0,InfoVis,2011,D³ Data-Driven Documents,10.1109/TVCG.2011.185,http://dx.doi.org/10.1109/TVCG.2011.185,2301,2309,J,Data-Driven Documents (D3) is a novel represen...,Michael Bostock;Vadim Ogievetsky;Jeffrey Heer,Michael Bostock;Vadim Ogievetsky;Jeffrey Heer,NaN,10.1109/INFVIS.2000.885091;10.1109/INFVIS.2000...,"Information visualization, user interfaces, to...",1537.0,1197.0,41.0,NaN
3,InfoVis,2006,Hierarchical Edge Bundles: Visualization of Ad...,10.1109/TVCG.2006.147,http://dx.doi.org/10.1109/TVCG.2006.147,741,748,J,A compound graph is a frequently encountered t...,Danny Holten,Danny Holten,NaN,10.1109/INFVIS.2004.1;10.1109/INFVIS.2003.1249...,"Network visualization, edge bundling, edge agg...",700.0,507.0,33.0,TT;BP
4,Vis,1997,ROAMing terrain: Real-time Optimally Adapting ...,10.1109/VISUAL.1997.663860,http://dx.doi.org/10.1109/VISUAL.1997.663860,81,88,C,Terrain visualization is a difficult problem f...,Mark A. Duchaineau;Murray Wolinsky;David E. Si...,M. Duchaineau;M. Wolinsky;D.E. Sigeti;M.C. Mil...,"Los Alamos Nat. Lab., NM, USA",10.1109/VISUAL.1996.567600;10.1109/VISUAL.1996...,"triangle bintree, view-dependent mesh, frame-t...",615.0,207.0,19.0,NaN
5,Vis,2003,Acceleration techniques for GPU-based volume r...,10.1109/VISUAL.2003.1250384,http://dx.doi.org/10.1109/VISUAL.2003.1250384,287,292,C,"Nowadays, direct volume rendering via 3D textu...",Jens H. Krüger;Rüdiger Westermann,J. Kruger;R. Westermann,"Comput. Graphics & Visualization Group, Tech. ...",10.1109/VISUAL.1999.809889;10.1109/VISUAL.1997...,"Volume Rendering, Programmable Graphics Hardwa...",606.0,247.0,16.0,TT
7,InfoVis,2005,Vizster: visualizing online social networks,10.1109/INFVIS.2005.1532126,http://dx.doi.org/10.1109/INFVIS.2005.1532126,32,39,C,Recent years have witnessed the dramatic popul...,Jeffrey Heer;Danah Boyd,J. Heer;D. Boyd,NaN,10.1109/INFVIS.2004.1,"social networks, visualization, graphs, commun...",592.0,97.0,21.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3389,VAST,2020,Visual Analytics of Multivariate Event Sequenc...,10.1109/VAST50239.2020.00009,http://dx.doi.org/10.1109/VAST50239.2020.00009,36,47,C,"In this work, we propose a generic visual anal...",Jiang Wu;Ziyang Guo;Zuobin Wang;Qingyang Xu;Yi...,Jiang Wu;Ziyang Guo;Zuobin Wang;Qingyang Xu;Yi...,"Zhejiang University,The State Key Lab of CAD&C...","10.1109/TVCG.2019.2934209,10.1109/TVCG.2017.27...","Sports Analytics,Event Sequence,Multivariate D...",NaN,0.0,55.0,NaN
3390,VAST,2020,Visual Causality Analysis of Event Sequence Data,10.1109/TVCG.2020.3030465,http://dx.doi.org/10.1109/TVCG.2020.3030465,1343,1352,J,Causality is crucial to understanding the mech...,Zhuochen Jin;Shunan Guo;Nan Chen;Daniel Weisko...,Zhuochen Jin;Shunan Guo;Nan Chen;Daniel Weisko...,iDVx Lab at Tongji University;iDVx Lab at Tong...,"10.1109/INFVIS.2003.1249025,10.1109/TVCG.2014....","Event sequence data,causality analysis,visual ...",NaN,0.0,53.0,NaN
3391,VAST,2020,Visual cohort comparison for spatial single-ce...,10.1109/TVCG.2020.3030336,http://dx.doi.org/10.1109/TVCG.2020.3030336,733,743,J,Spatially-resolved omics-data enable researche...,Antonios Somarakis;Marieke E. Ijsselsteijn;Sie...,Antonios Somarakis;Marieke E. Ijsselsteijn;Sie...,"Department of Radiology, Division of Image Pro...","10.1109/TVCG.2013.124,10.1109/TVCG.2018.286490...","Visual analytics,Imaging Mass Cytometry,Vectra...",NaN,0.0,58.0,NaN
3392,VAST,2020,Visual Neural Decomposition to Explain Multiva...,10.1109/TVCG.2020.3030420,http://dx.doi.org/10.1109/TVCG.2020.3030420,1374,1384,J,Investigating relationships between variables ...,Johannes Knittel;Andres Lalama;Steffen Koch;Th...,Johannes Knittel;Andres Lalama;Steffen Koch;Th...,University of Stuttgart;University of Stuttgar...,"10.1109/

In [4]:
# Splitting keywords
keywords = []


def remove_prefix(text, prefix):
    if text.startswith(prefix):
        return text[len(prefix):]
    return text


def split(elem):
    global keywords
    elem = elem.replace(";", ",")
    res = []

    if "," in elem:
        res = [remove_prefix(x.lower().strip(), 'and ')
               for x in elem.split(",")]
    elif "-" in elem:
        res = [remove_prefix(x.lower().strip(), 'and ')
               for x in elem.split("-")]
    elif elem != "":
        res = [remove_prefix(elem.lower().strip(), 'and ')]

    keywords.extend(res)
    return res


data['AuthorKeywords'] = data["AuthorKeywords"].parallel_appy(split)
data


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Conference,Year,Title,DOI,Link,FirstPage,LastPage,PaperType,Abstract,AuthorNames-Deduped,AuthorNames,AuthorAffiliation,InternalReferences,AuthorKeywords,AminerCitationCount_04-2020,XploreCitationCount - 2021-02,PubsCited,Award
0,InfoVis,2011,D³ Data-Driven Documents,10.1109/TVCG.2011.185,http://dx.doi.org/10.1109/TVCG.2011.185,2301,2309,J,Data-Driven Documents (D3) is a novel represen...,Michael Bostock;Vadim Ogievetsky;Jeffrey Heer,Michael Bostock;Vadim Ogievetsky;Jeffrey Heer,NaN,10.1109/INFVIS.2000.885091;10.1109/INFVIS.2000...,"[information visualization, user interfaces, t...",1537.0,1197.0,41.0,NaN
3,InfoVis,2006,Hierarchical Edge Bundles: Visualization of Ad...,10.1109/TVCG.2006.147,http://dx.doi.org/10.1109/TVCG.2006.147,741,748,J,A compound graph is a frequently encountered t...,Danny Holten,Danny Holten,NaN,10.1109/INFVIS.2004.1;10.1109/INFVIS.2003.1249...,"[network visualization, edge bundling, edge ag...",700.0,507.0,33.0,TT;BP
4,Vis,1997,ROAMing terrain: Real-time Optimally Adapting ...,10.1109/VISUAL.1997.663860,http://dx.doi.org/10.1109/VISUAL.1997.663860,81,88,C,Terrain visualization is a difficult problem f...,Mark A. Duchaineau;Murray Wolinsky;David E. Si...,M. Duchaineau;M. Wolinsky;D.E. Sigeti;M.C. Mil...,"Los Alamos Nat. Lab., NM, USA",10.1109/VISUAL.1996.567600;10.1109/VISUAL.1996...,"[triangle bintree, view-dependent mesh, frame-...",615.0,207.0,19.0,NaN
5,Vis,2003,Acceleration techniques for GPU-based volume r...,10.1109/VISUAL.2003.1250384,http://dx.doi.org/10.1109/VISUAL.2003.1250384,287,292,C,"Nowadays, direct volume rendering via 3D textu...",Jens H. Krüger;Rüdiger Westermann,J. Kruger;R. Westermann,"Comput. Graphics & Visualization Group, Tech. ...",10.1109/VISUAL.1999.809889;10.1109/VISUAL.1997...,"[volume rendering, programmable graphics hardw...",606.0,247.0,16.0,TT
7,InfoVis,2005,Vizster: visualizing online social networks,10.1109/INFVIS.2005.1532126,http://dx.doi.org/10.1109/INFVIS.2005.1532126,32,39,C,Recent years have witnessed the dramatic popul...,Jeffrey Heer;Danah Boyd,J. Heer;D. Boyd,NaN,10.1109/INFVIS.2004.1,"[social networks, visualization, graphs, commu...",592.0,97.0,21.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3389,VAST,2020,Visual Analytics of Multivariate Event Sequenc...,10.1109/VAST50239.2020.00009,http://dx.doi.org/10.1109/VAST50239.2020.00009,36,47,C,"In this work, we propose a generic visual anal...",Jiang Wu;Ziyang Guo;Zuobin Wang;Qingyang Xu;Yi...,Jiang Wu;Ziyang Guo;Zuobin Wang;Qingyang Xu;Yi...,"Zhejiang University,The State Key Lab of CAD&C...","10.1109/TVCG.2019.2934209,10.1109/TVCG.2017.27...","[sports analytics, event sequence, multivariat...",NaN,0.0,55.0,NaN
3390,VAST,2020,Visual Causality Analysis of Event Sequence Data,10.1109/TVCG.2020.3030465,http://dx.doi.org/10.1109/TVCG.2020.3030465,1343,1352,J,Causality is crucial to understanding the mech...,Zhuochen Jin;Shunan Guo;Nan Chen;Daniel Weisko...,Zhuochen Jin;Shunan Guo;Nan Chen;Daniel Weisko...,iDVx Lab at Tongji University;iDVx Lab at Tong...,"10.1109/INFVIS.2003.1249025,10.1109/TVCG.2014....","[event sequence data, causality analysis, visu...",NaN,0.0,53.0,NaN
3391,VAST,2020,Visual cohort comparison for spatial single-ce...,10.1109/TVCG.2020.3030336,http://dx.doi.org/10.1109/TVCG.2020.3030336,733,743,J,Spatially-resolved omics-data enable researche...,Antonios Somarakis;Marieke E. Ijsselsteijn;Sie...,Antonios Somarakis;Marieke E. Ijsselsteijn;Sie...,"Department of Radiology, Division of Image Pro...","10.1109/TVCG.2013.124,10.1109/TVCG.2018.286490...","[visual analytics, imaging mass cytometry, vec...",NaN,0.0,58.0,NaN
3392,VAST,2020,Visual Neural Decomposition to Explain Multiva...,10.1109/TVCG.2020.3030420,http://dx.doi.org/10.1109/TVCG.2020.3030420,1374,1384,J,Investigating relationships between variables ...,Johannes Knittel;Andres Lalama;Steffen Koch;Th...,Johannes Knittel;Andres Lalama;Steffen Koch;Th...,University of Stuttgart;University of Stuttgar...,"10.1109/

In [11]:
# sorting and removing duplicates
keywords = list(set(keywords))
keywords.sort()
keywordsDf = pd.DataFrame(keywords)
keywordsDf


,0
0,2d
1,2d and 3d visualization
2,2d asymmetric tensor field topology
3,2d asymmetric tensor fields
4,2d graphics
...,...
5675,zoomable grid
5676,zoomable treemaps
5677,zoomable user interfaces (zuis)
5678,zooming


In [6]:
# extracting keywords with full text and acronyms
p = re.compile('.*[(].*[)]')
acr = [k for k in keywords if p.match(k)]
acrList = [sorted(k.replace(")", "").replace(
    "methods", '').split("("), key=len) for k in acr]
acrList1 = [k[0].strip() for k in acrList]
acrList2 = [k[1].strip() for k in acrList]
acrDf = pd.DataFrame()
acrDf[0] = acr
acrDf[1] = acrList1
acrDf[2] = acrList2
acrDf


,0,1,2
0,computational fluid dynamics (cfd),cfd,computational fluid dynamics
1,computed tomography angiography (cta),cta,computed tomography angiography
2,continuous galerkin (cg) methods,cg,continuous galerkin
3,deep q-network (dqn),dqn,deep q-network
4,discontinuous galerkin (dg) methods,dg,discontinuous galerkin
5,explainable artificial intelligence (xai),xai,explainable artificial intelligence
6,extraction of surfaces((isosurfaces),,isosurfaces
7,finite element methods (fem),fem,finite element
8,finite-difference-time-domain (fdtd),fdtd,finite-difference-time-domain
9,gaussian mixture model (gmm),gmm,gaussian mixture model


In [7]:
# removing non acronyms
acrDf = acrDf.drop([6, 11, 15, 16, 22])
acrDf


,0,1,2
0,computational fluid dynamics (cfd),cfd,computational fluid dynamics
1,computed tomography angiography (cta),cta,computed tomography angiography
2,continuous galerkin (cg) methods,cg,continuous galerkin
3,deep q-network (dqn),dqn,deep q-network
4,discontinuous galerkin (dg) methods,dg,discontinuous galerkin
5,explainable artificial intelligence (xai),xai,explainable artificial intelligence
7,finite element methods (fem),fem,finite element
8,finite-difference-time-domain (fdtd),fdtd,finite-difference-time-domain
9,gaussian mixture model (gmm),gmm,gaussian mixture model
10,human phenotype ontology (hpo),hpo,human phenotype ontology


In [12]:
def replace(y):
    new1 = acrDf.loc[acrDf[0] == y][2]
    new2 = acrDf.loc[acrDf[1] == y][2]
    if not new1.empty:
        return list(new1)[0]
    if not new2.empty:
        return list(new2)[0]
    else:
        return y


data["AuthorKeywords"] = data["AuthorKeywords"].parallel_apply(
    (lambda x: list(map(replace, x))))
keywordsDf[0] = keywordsDf[0].parallel_apply(replace)
keywordsDf


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,0
0,2d
1,2d and 3d visualization
2,2d asymmetric tensor field topology
3,2d asymmetric tensor fields
4,2d graphics
...,...
5675,zoomable grid
5676,zoomable treemaps
5677,zoomable user interfaces
5678,zooming


In [18]:
stop_words = set(stopwords.words('english'))
'''
def process(s):
    token_words=[]

    for word in word_tokenize(s.replace('-', '').replace("&", "and")):
            splitted = splitter.split(word)
            splitted = splitted if splitted!='' else [word]
            for word in splitted:
                if word not in string.punctuation and word not in stop_words:
                   token_words.append(word) 

    s = s.replace('-', ' ').replace("&", "and").replace("+", " ")
    splitted = splitter.split(s.replace(" ", ''))
    splitted = splitted if len(splitted) > 0 else word_tokenize(s)
    for word in splitted:
        if word not in string.punctuation and word not in stop_words:
                token_words.append(word)

    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(PorterStemmer().stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)
'''


def process1(s):
    token_words = []

    for word in word_tokenize(s.replace('-', '').replace("&", "and").replace("+", " ")):
        splitted = splitter.split(word)
        splitted = splitted if splitted != '' else [word]
        for word in splitted:
            if word not in string.punctuation and word not in stop_words:
                token_words.append(word)

    stem_sentence = []
    for word in token_words:
        stem_sentence.append(PorterStemmer().stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

def process2(s):
    token_words = []

    for word in word_tokenize(s.replace('-', '').replace("&", "and").replace("+", " ")):
        if word not in string.punctuation and word not in stop_words:
            token_words.append(word)

    stem_sentence = []
    for word in token_words:
        stem_sentence.append(PorterStemmer().stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)


In [19]:
keywordsDf[1] = keywordsDf[0].parallel_apply(process1)
keywordsDf


,0,1
0,2d,2d
1,2d and 3d visualization,2d 3d visual
2,2d asymmetric tensor field topology,2d asymmetr tensor field topolog
3,2d asymmetric tensor fields,2d asymmetr tensor field
4,2d graphics,2d graphic
...,...,...
5675,zoomable grid,zoom abl grid
5676,zoomable treemaps,zoom abl tree map
5677,zoomable user interfaces,zoom abl user interfac
5678,zooming,zoom


In [21]:
grouped = keywordsDf.groupby([1])[0].apply(list).reset_index()
grouped[1] = grouped[0].parallel_apply(lambda x: x[0])
grouped


,1,0
0,maup,[maup]
1,2d,[2d]
2,2d and 3d visualization,[2d and 3d visualization]
3,2d asymmetric tensor fields,[2d asymmetric tensor fields]
4,2d asymmetric tensor field topology,[2d asymmetric tensor field topology]
...,...,...
5246,zoom,"[zoom, zooming]"
5247,zoomable grid,[zoomable grid]
5248,zoomable treemaps,[zoomable treemaps]
5249,zoomable user interfaces,[zoomable user interfaces]


In [22]:
exploded = grouped.explode(0)
exploded

,1,0
0,maup,maup
1,2d,2d
2,2d and 3d visualization,2d and 3d visualization
3,2d asymmetric tensor fields,2d asymmetric tensor fields
4,2d asymmetric tensor field topology,2d asymmetric tensor field topology
...,...,...
5246,zoom,zooming
5247,zoomable grid,zoomable grid
5248,zoomable treemaps,zoomable treemaps
5249,zoomable user interfaces,zoomable user interfaces


In [23]:
def replace(y):
    new = exploded.loc[exploded[0] == y][1]
    if not new.empty:
        return list(new)[0]
    else:
        return y


data["AuthorKeywords"] = data["AuthorKeywords"].parallel_apply(
    (lambda x: list(map(replace, x))))
keywordsDf[0] = keywordsDf[0].parallel_apply(replace)
keywordsDf

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,0,1
0,2d,2d
1,2d and 3d visualization,2d 3d visual
2,2d asymmetric tensor field topology,2d asymmetr tensor field topolog
3,2d asymmetric tensor fields,2d asymmetr tensor field
4,2d graphics,2d graphic
...,...,...
5675,zoomable grid,zoom abl grid
5676,zoomable treemaps,zoom abl tree map
5677,zoomable user interfaces,zoom abl user interfac
5678,zoom,zoom


In [25]:
keywordsDf[1] = keywordsDf[0].parallel_apply(process2)
grouped = keywordsDf.groupby([1])[0].apply(list).reset_index()
grouped[1] = grouped[0].parallel_apply(lambda x: x[0])
exploded = grouped.explode(0)
def replace(y):
    new = exploded.loc[exploded[0] == y][1]
    if not new.empty:
        return list(new)[0]
    else:
        return y


data["AuthorKeywords"] = data["AuthorKeywords"].parallel_apply(
    (lambda x: list(map(replace, x))))
keywordsDf[0] = keywordsDf[0].parallel_apply(replace)
keywordsDf

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,0,1
0,2d,2d
1,2d and 3d visualization,2d 3d visual
2,2d asymmetric tensor field topology,2d asymmetr tensor field topolog
3,2d asymmetric tensor fields,2d asymmetr tensor field
4,2d graphics,2d graphic
...,...,...
5675,zoomable grid,zoomabl grid
5676,zoomable treemaps,zoomabl treemap
5677,zoomable user interfaces,zoomabl user interfac
5678,zoom,zoom


In [35]:
def replace_paper(p):
    types = {'C': 'Conference paper', 'J': 'Journal paper', 'M': 'Miscellaneous'}
    return types[p]

cleaned_data = data.reindex(columns=['DOI', 'Title', 'Conference'])
cleaned_data["PaperType"] = data["PaperType"].parallel_apply(replace_paper)
cleaned_data["Year"] = data["Year"].astype(int)
cleaned_data["Authors"] = data["AuthorNames-Deduped"]
cleaned_data['InternalReferences'] = data['InternalReferences'].fillna('').apply(lambda x: x.replace(',', ';'))
cleaned_data["Keywords"] = data["AuthorKeywords"].apply(lambda x: ";".join(x))
cleaned_data.to_csv("../preprocessed-data/papers.csv", sep=",", index=False)
cleaned_data

,DOI,Title,Conference,PaperType,Year,Authors,InternalReferences,Keywords
0,10.1109/TVCG.2011.185,D³ Data-Driven Documents,InfoVis,Journal paper,2011,Michael Bostock;Vadim Ogievetsky;Jeffrey Heer,10.1109/INFVIS.2000.885091;10.1109/INFVIS.2000...,information visualization;user interface;toolk...
3,10.1109/TVCG.2006.147,Hierarchical Edge Bundles: Visualization of Ad...,InfoVis,Journal paper,2006,Danny Holten,10.1109/INFVIS.2004.1;10.1109/INFVIS.2003.1249...,network visualization;edge bundles;edge aggreg...
4,10.1109/VISUAL.1997.663860,ROAMing terrain: Real-time Optimally Adapting ...,Vis,Conference paper,1997,Mark A. Duchaineau;Murray Wolinsky;David E. Si...,10.1109/VISUAL.1996.567600;10.1109/VISUAL.1996...,triangle bintree;view-dependent mesh;frame-to-...
5,10.1109/VISUAL.2003.1250384,Acceleration techniques for GPU-based volume r...,Vis,Conference paper,2003,Jens H. Krüger;Rüdiger Westermann,10.1109/VISUAL.1999.809889;10.1109/VISUAL.1997...,volume rendering;programmable graphics hardwar...
7,10.1109/INFVIS.2005.1532126,Vizster: visualizing online social networks,InfoVis,Conference paper,2005,Jeffrey Heer;Danah Boyd,10.1109/INFVIS.2004.1,social network;visualization;graph;community;d...
...,...,...,...,...,...,...,...,...
3389,10.1109/VAST50239.2020.00009,Visual Analytics of Multivariate Event Sequenc...,VAST,Conference paper,2020,Jiang Wu;Ziyang Guo;Zuobin Wang;Qingyang Xu;Yi...,10.1109/TVCG.2019.2934209;10.1109/TVCG.2017.27...,sport analytics;event sequence;multi-variate d...
3390,10.1109/TVCG.2020.3030465,Visual Causality Analysis of Event Sequence Data,VAST,Journal paper,2020,Zhuochen Jin;Shunan Guo;Nan Chen;Daniel Weisko...,10.1109/INFVIS.2003.1249025;10.1109/TVCG.2014....,event sequence data;causality analysis;visual ...
3391,10.1109/TVCG.2020.3030336,Visual cohort comparison for spatial single-ce...,VAST,Journal paper,2020,Antonios Somarakis;Marieke E. Ijsselsteijn;Sie...,10.1109/TVCG.2013.124;10.1109/TVCG.2018.286490...,visual analytic;imaging mass cytometry;vectra;...
3392,10.1109/TVCG.2020.3030420,Visual Neural Decomposition to Explain Multiva...,VAST,Journal paper,2020,Johannes Knittel;Andres Lalama;Steffen Koch;Th...,10.1109/INFVIS.2004.68;10.1109/VAST.2008.46773...,visual analytic;multivariate data analysis;mac...
